In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
#from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN


In [6]:
def neighbors_search(data, point, eps, metric):
    """
    Generates neighborhood graph for a point
    """
    neighbors = []
    S=0
    for i in range(data.shape[0]):
        if metric(data[point], data[i]) < eps:
            S+=1
            neighbors.append(i)
    
    return neighbors


def simple_DBSCAN(data, clust, eps, minPts, metric=distance.euclidean):
    """
    It iterates through neighbors_search for every point in data
    expands cluster for every point not determined to be noise
    """
    current_point = 0
    
    for i in range(0, data.shape[0]):
        if clust[i] != 0:
            continue
    
        neighbors = neighbors_search(data, i, eps, metric)

        if len(neighbors) < minPts:
            clust[i] = -1

        else:
            current_point += 1
            expand(data, clust, i, neighbors, current_point, eps, minPts, metric)
    
    return clust


def expand(data, clust, point, neighbors, current_point, eps, minPts, metric):
    """
    Expands cluster from a given point until neighborhood boundaries are reached
    """
    clust[point] = current_point
    
    i = 0
    while i < len(neighbors):
        
        nextPoint = neighbors[i]
        
        if clust[nextPoint] == -1:
            clust[nextPoint] = current_point
        
        elif clust[nextPoint] == 0:
            clust[nextPoint] = current_point
            
            next_neighbors = neighbors_search(data, nextPoint, eps, metric)
            
            if len(next_neighbors) >= minPts:
                neighbors = neighbors + next_neighbors
                
        i += 1


In [7]:
class Basic_DBSCAN:
    """
    Parameters:
    
    eps: Radius of neighborhood graph
    
    minPts: Mininmu number of neighbors required to label a given point as a core point.
    
    metric: Distance metric used to determine distance between points    
    """
    
    def __init__(self, eps, minPts, metric=distance.euclidean):
        self.eps = eps
        self.minPts = minPts
        self.metric = metric
    
    def fit_predict(self, X):
        """
        Parameters:
        
        X: An n-dimensional array of numeric vectors to be analyzed
        
        Returns:
        
        [n] cluster labels
        """
    
        clusters = [0] * X.shape[0]
        
        simple_DBSCAN(X, clusters, self.eps, self.minPts, self.metric)
        
        return clusters


In [9]:
df = pd.read_csv('minute_weather.csv') # ~10% of the original dataset
df = df.dropna() # dropping entries with missing values
df = df.head(2000) # taking the first 1000 entries; ~10% of the ~10%
df = df.drop(columns=['rowID', 'hpwren_timestamp', 'rain_accumulation', 'rain_duration']) # dropping unwanted columns
cols = df.columns

def checkEqual(l1, l2):
    """
    Helper to illustrate accuracy of results
    """
    return len(l1) == len(l2) and sorted(l1) == sorted(l2)


In [ ]:
X = df[[cols[0], cols[1]]]
X = StandardScaler().fit_transform(X)

scanner = Basic_DBSCAN(eps=0.3, minPts=30)
clusters = scanner.fit_predict(X)

sk_clusters = DBSCAN(eps=0.3, min_samples=30, metric=distance.euclidean).fit(X).labels_
sk_clusters = [x+1
               if x != -1
               else x
               for x in sk_clusters]

print(f'Results are same as sk-learn: {checkEqual(clusters, sk_clusters)}')

# my clusters
sns.scatterplot(df.iloc[:, 0], df.iloc[:, 1], 
                hue=clusters, palette='bright')
plt.title('Plotted from my clusters!')
plt.show();

# sk-learn clusters
sns.scatterplot(df.iloc[:, 0], df.iloc[:, 1], 
                hue=sk_clusters, palette='bright')
plt.title('Plotted from sk-learn clusters!')
plt.show();

In [ ]:
print()

In [62]:
import warnings
warnings.filterwarnings("ignore")                     #Ignoring unnecessory warnings

import numpy as np                                  #for large and multi-dimensional arrays
import pandas as pd                                 #for data manipulation and analysis
import nltk                                         #Natural language processing tool-kit

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer

from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF
from gensim.models import Word2Vec                                   #For Word2Vec

In [48]:
data_path = "input/Reviews.csv"
data = pd.read_csv(data_path)
data_sel = data.head(1000)                                #Considering only top 10000 rows
# Shape of our data
data_sel.columns


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [49]:
data_score_removed = data_sel[data_sel['Score']!=3]       #Neutral reviews removed

def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

score_upd = data_score_removed['Score']
t = score_upd.map(partition)
data_score_removed['Score']=t

In [50]:
final_data = data_score_removed.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})
final = final_data[final_data['HelpfulnessNumerator'] <= final_data['HelpfulnessDenominator']]
final_X = final['Text']
final_y = final['Score']

In [51]:
stop = set(stopwords.words('english')) 
print(stop)

import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in final_X:
    print(sentence)
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # Stemming and removing stopwords
    temp.append(words)
print("Z")
final_X = temp    
sent = []
for row in final_X:
    print(row)
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

final_X = sent
print(final_X[1])

{'are', 'the', 'same', "aren't", 'isn', 'our', 'don', 'because', 'up', 'this', 'yours', "mustn't", 'have', 'having', 'once', 'does', 'theirs', 'above', 'am', 'didn', 'needn', 'over', 'only', 'out', 'themselves', 'can', "that'll", 'll', 've', 'wasn', 'other', 'that', 'not', 'they', 'has', 'with', 'will', 'hasn', 'too', 'than', 'before', 'when', "weren't", 'was', 'why', 'which', 'yourselves', "should've", 'no', 'own', 'into', 'few', 'm', 'won', 'their', 'he', "won't", 'is', 'yourself', 'aren', 'it', 'very', "she's", 'but', 'shan', 'who', 'i', 'by', 'down', 'so', 'myself', 'be', 'such', 'and', 'hers', 'mustn', "shouldn't", "you've", 'shouldn', 'do', 's', 'how', 'ourselves', 'to', 'haven', 'were', 'a', 't', 'under', "you'd", 'as', 'on', 'there', "don't", "mightn't", 'me', 'herself', 'she', 'ain', 'from', "hadn't", 'whom', 'at', 'been', 'weren', 'its', 'most', 'now', "didn't", 'of', "you're", 'or', 'an', 'through', 'himself', "you'll", 'd', 'should', "wouldn't", 'wouldn', 'between', "it's",

I roast at home with a stove-top popcorn popper (but I do it outside, of course). These beans (Coffee Bean Direct Green Mexican Altura) seem to be well-suited for this method. The first and second cracks are distinct, and I've roasted the beans from medium to slightly dark with great results every time. The aroma is strong and persistent. The taste is smooth, velvety, yet lively.
We roast these in a large cast iron pan on the grill (about 1/3 of the bag at a time).  The smell is wonderful and the roasted beans taste delicious too.  More importantly, the coffee is smooth; no bitter aftertaste.  On numerous occasions, we've had to send the roasted beans home with friends because they like it so much.
Deal was awesome!  Arrived before Halloween as indicated and was enough to satisfy trick or treaters.  I love the quality of this product and it was much less expensive than the local store's candy.
It is chocolate, what can I say.  Great variety of everything our family loves.  With a famil

Great gift for all ages! I purchased these giant canes before and the recipients loved them so much, they kept them and would not eat them.
I know the product title says Molecular Gastronomy, but don't let that scare you off.  I have been looking for this for a while now, not for food science, but for something more down to earth.  I use it to make my own coffee creamer.<br /><br />I have to have my coffee blonde and sweet - but the flavored creamers are full of the bad kinds of fat, and honestly, I hate to use manufactured "food" items.  I really don't think they are good for the body.  On the other hand, I hate using cold milk or cream, because I like HOT coffee.<br /><br />I stumbled across this on Amazon one day and got the idea of making my own creamer.  I also bought low-fat (non-instant) milk powder and regular milk powder. The non-instant lowfat milk is a little sweeter and tastes fresher than regular instant low-fat milk, but does not dissolve good in cold water - which is not

These are absolutely scrumptuous!  My husband and I both love them, however, as another customer put it, they are expensive to ship!  The cost of shipping is more than the tartlets themselves are!
What a nice alternative to an apple pie. Love the fact there was no slicing and dicing. Easy to prepare. I also loved the fact that you can make them fresh whenever needed.
I like Creme Brulee. I loved that these were so easy. Just sprinkle on the sugar that came with and broil. They look amazing and taste great. My guess thought I really went out of the way for them when really it took all of 5 minutes. I will be ordering more!
I first bought pickled asparagus at an Amish market and it was love at first bite. Hence, when I saw these on the discount rack at the local Safeway I cleaned it out.  These are even better than the ones from the Amish market. They are fantastic, and some of the jars have a bonus clove of garlic at the bottom. I'm going to order the six pack and I'm a household of one

My holistic vet recommended this, along with a few other brands. We tried them all, but my cats prefer this (especially the sardine version). The best part is their coats are so soft and clean and their eyes are so clear. AND (and I don't want to be rude, so I'll say this as delicately as I can) their waste is far less odorous than cats who eat the McDonalds junk found in most stores, which is a definite plus for me! The health benefits are so obvious - I highly recommend Holistic Select!
One of my cats is allergic to fish and beef. This formula is one of the few she can eat, and it has much better ingredients than the prescription diets available at the vet. Both of my kitties are very active, have soft shiny fur, and neither are fat. Dry food reduces tartar buildup on teeth, also.
Our cats thrive extremely well on this dry cat food.  They definitely have much less hair ball throw ups and their fur is great.  They are fit and not over weight.  This vendor ships extremely fast.  Is one

...

Refer to the picture I posted above.  The bar is half the size you think it is going to be...It is literally a two-bite bar.  Very deceptive advertising.  That and the fact that they are ridiculously expensive is the reason for the one star.  My 10 year old son ate one and didn't like them. I ate one and agreed as did my husband.  Couldn't give it away!  Everyone has different tastes but these are no tasty treat as far as I'm concerned.
With all the allergies in our family, these ended up working out great.  We don't have any negative reactions to the ingredients and though we don't buy many pre packaged foods, these work for 'on-the-go'. Our family tried several varieties and I prefer the Cocoa Loco Bars; they're almost like a Tootsie Roll to me-though I can't eat Tootsie Rolls, so this is a great substitute for that occasional craving.  My family prefers the Very Berry Bars because they can eat those when they miss breakfast, need a healthy snack, or my son will take them on his cros

Baby loved this one, but as he has progressed I had to cut this one out because it is too runny and if I have to move the spoon quickly out of the way of his grabby hands, it is all over! I think they could use a little less water maybe a little more potato to make it more the consistency of the turkey dinner one.
My little girl can't get enough of this! I like that it is organic and contains fruit, veggies, protein and grains in one tasty meal. I wish it came in larger portions since my 10 month old always wants 2 jars.
Its my daughter's favorite first and foremost.  For me I love that i can get lentils, there aren't many stage 2 or 3 vegetarian choices.
Wow, i am shocked to find clear plastic pieces in the jars. The other posters say they notified Earth's Best about this in August and they are still selling it without checking or recalling!! No more earth's best for my little guy. I am taking my business elsewhere and I am very sad that organic company is selling baby food with plast

My son started eating this when he was around 8 months old; he's now 14 months old and it's one of his favorites. Unfortunately, they don't carry it at the local grocery store (they carry other varieties of Earth's Best; just not this one or his other favorites, Spinach/Potatoes and Spaghetti with Cheese); they do carry it at some of the Whole Foods so I buy them there. I'm so glad to find them on Amazon! I get TIRED of ringing up a gazillion little jars at the store.<br /><br />We use this as a base and mix in other things - bits of cooked broccoli, small pieces of meat. I've also started adding spices to introduce him to new flavors.
My baby loves Rice and Lentil and is one of her favorite things to eat.  If we are going somewhere I won't be able to warm the food she will eat this one cold.
My daughter loved this stuff. It is all natural and made very well.  Good to mix with other things as they get older too!  Beats Gerber.
My daughter is outgrowing baby food but will still eat this

Best paste I have tried! So easy to use. Just saute you meat, add veggies, paste, and a tin of coconut milk. Quick and tasty! I am on weight watchers and tried using the light coconut milk instead of the full fat. It works for the panang paste especially if you add a heaping tablespoon of crunchy peanut butter.. but the yellow, green, and red are spicy and need the full fat coconut milk to balance it out.
OMG I would love these cookies except for the chocolate mixed in makes it taste medicinal. DEGUSTING isn't even a strong enough word. A horrible horrible mistake...an expensive mistake....they advertised it as peanut butter cookies which is why I bought them since everyone else had peanut butter chocolate cookies when I just wanted plan peanut butter cookies....however when I received them I found out they were the chocolate peanut butter ones but I thought okay I'll try them and omg I almost threw up from the taste. HORRIBLE, HORRIBLE, DISGUTINGLY, MEDICINAL TASTE. DO NOT BUY!!! If t

This is my favorite hot sauce! I liketo use it to give anything some extra flavor. I have loved going out to some of my favorite resturants in town and seeing that they are offering this great product.I really enjoy putting on my my breakfast burritos!
They have this in a local diner that I eat in regularly.  I use it on all kinds of foods and I love it.  It's hot, but not without flavor.  I don't like those sauces that just burn but have no taste.  This has both burn and and awesome flavor.  I was surprised to find it on Amazon (just stumbled across it).  I would highly recommend it to anyone who likes a good hot sauce.
I was browsing around on Amazon looking for gifts and was very pleased to see my favorite sauce on here. I will just order and have shipped to my friends. Habanero is my favorite, I love the heat and the tangy spiciness. I also love the pineapple. You will not go wrong ordering this up.
This is my favorite hot sauce. I buy it locally and love the flavor that they all h

In [52]:
tf_w_data = final_X
tf_idf = TfidfVectorizer(max_features=5000)
tf_idf_data = tf_idf.fit_transform(tf_w_data)
tf_w_data = []
tf_idf_data = tf_idf_data.toarray()
i = 0
w2v_data = final_X
splitted = []
for row in w2v_data: 
    splitted.append([word for word in row.split()])     #splitting words*
train_w2v = Word2Vec(splitted,min_count=5,size=50, workers=4)
avg_data = []
for row in splitted:
    vec = np.zeros(50)
    count = 0
    for word in row:
        try:
            vec += train_w2v[word]
            count += 1
        except:
            pass
    avg_data.append(vec/count)
print(avg_data[1])
   
for row in splitted:
    vec = [0 for i in range(50)]
    
    temp_tfidf = []
    for val in tf_idf_data[i]:
        if val != 0:
            temp_tfidf.append(val)
    
    count = 0
    tf_idf_sum = 0
    for word in row:
        try:
            count += 1
            tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
            vec += (temp_tfidf[count-1] * train_w2v[word])
        except:
            pass
    vec = (float)(1/tf_idf_sum) * vec
    tf_w_data.append(vec)
    i = i + 1

print(tf_w_data[1])
    

[-4.19246043e-01 -1.09510085e-01 -1.08248367e-01  4.29903665e-01
 -1.56712485e-01 -9.13933072e-04 -3.20639119e-02  2.27803597e-01
 -3.14055404e-01  4.47426213e-01  1.59102294e-01 -2.87538567e-01
 -2.44771688e-01 -9.08676743e-02  2.94625048e-01  1.41879533e-01
 -3.40490704e-01  3.71209309e-01 -6.31305775e-02 -1.59081495e-01
  9.96683618e-02  9.24133014e-01 -6.11369205e-01 -1.16146743e-02
 -3.68607587e-01 -3.97767590e-01  5.72910694e-01 -5.18153207e-01
 -4.35007330e-01 -2.98786760e-01  2.18593931e-01 -2.22381172e-01
 -2.72650626e-01  2.38751158e-01 -1.63678087e-01  2.97116194e-01
  2.88937453e-02  1.13423067e-01 -2.10725276e-01  3.46325632e-01
  4.42720800e-01 -5.45585091e-01 -6.13289944e-03  2.95613800e-01
  6.76268793e-01  2.29046227e-01 -7.15805896e-02 -2.34561895e-01
 -2.05605918e-01 -1.45084512e-01]
[-0.3361525  -0.08655798 -0.08610927  0.34187905 -0.12643225 -0.00103542
 -0.02471133  0.18273582 -0.25226525  0.35909592  0.12791015 -0.2329764
 -0.19657099 -0.0741859   0.23500173  0.1

In [53]:
tf_w_data = final_X
tf_idf = TfidfVectorizer(max_features=5000)
tf_idf_data = tf_idf.fit_transform(tf_w_data)
tf_w_data = []
tf_idf_data = tf_idf_data.toarray()
i = 0
for row in splitted:
    vec = [0 for i in range(50)]
    
    temp_tfidf = []
    for val in tf_idf_data[i]:
        if val != 0:
            temp_tfidf.append(val)
    
    count = 0
    tf_idf_sum = 0
    for word in row:
        try:
            count += 1
            tf_idf_sum = tf_idf_sum + temp_tfidf[count-1]
            vec += (temp_tfidf[count-1] * train_w2v[word])
        except:
            pass
    vec = (float)(1/tf_idf_sum) * vec
    tf_w_data.append(vec)
    i = i + 1

print(tf_w_data[1])

[-0.3361525  -0.08655798 -0.08610927  0.34187905 -0.12643225 -0.00103542
 -0.02471133  0.18273582 -0.25226525  0.35909592  0.12791015 -0.2329764
 -0.19657099 -0.0741859   0.23500173  0.11482674 -0.27288453  0.29925726
 -0.05042225 -0.12587429  0.08086012  0.73934929 -0.48891517 -0.00897745
 -0.29597868 -0.316759    0.45955976 -0.41687394 -0.34918753 -0.23816306
  0.17401531 -0.17932271 -0.21939967  0.18982678 -0.1305189   0.23822516
  0.02433846  0.09129596 -0.16686148  0.27640056  0.3558326  -0.43703585
 -0.00393253  0.23915026  0.54321013  0.18254435 -0.05967074 -0.1878649
 -0.16584062 -0.11598725]


In [101]:
tf_w_datat

[-0.008352594087162872,
 -0.007843048154066803,
 -0.005552844619694694,
 -0.00791988955256256,
 -0.006240122977902867,
 -0.0070277940696971,
 -0.007413973090273914,
 -0.009939370770444204,
 -0.0053572413332771875,
 -0.011626372257583537,
 -0.007867127493359377,
 -0.00812972831725407,
 -0.00911960884616558,
 -0.011165507384039357,
 -0.0052743718814106865,
 -0.007268322066627281,
 -0.00967887420725612,
 -0.008723475144231812,
 -0.005524801425470274,
 -0.008995978583066316,
 -0.007971260961996305,
 -0.006657571445084752,
 -0.008611505906941931,
 -0.008240643057345218,
 -0.005671928794712675,
 -0.008860062724908847,
 -0.011126319736207526,
 -0.00882205313691591,
 -0.007579102468325667,
 -0.007637142721976921,
 -0.012345317917545949,
 -0.00997432446484598,
 -0.009419312557528356,
 -0.00775688134138598,
 -0.007966310831118676,
 -0.0105092033169685,
 -0.007542186259624775,
 -0.006697300983013252,
 -0.008819628035766372,
 -0.008138942051103637,
 -0.008283189234027422,
 -0.009355090107073905,
 

In [121]:
tf_w_datat=[i.mean() for i in tf_w_data]
X = np.array(tf_w_data)
Xt = np.array(tf_w_datat)

scanner = Basic_DBSCAN(eps=0.1, minPts=50)
clusters = scanner.fit_predict(X)

sk_clusters = DBSCAN(eps=0.10, min_samples=30, metric=distance.euclidean).fit(X).labels_
sk_clusters = [x+1
               if x != -1
               else x
               for x in sk_clusters]

print(f'Results are same as sk-learn: {checkEqual(clusters, sk_clusters)}')

172
179
180
162
178
179
180
173
165
160
171
174
166
174
172
164
173
167
160
165
173
173
177
162
160
169
172
175
167
174
175
164
174
166
176
166
159
162
167
165
171
177
181
175
175
174
174
173
171
165
171
160
174
167
180
168
162
178
165
175
173
164
174
173
171
179
168
172
158
178
179
162
165
180
176
180
162
172
169
168
166
162
162
176
162
179
176
163
166
163
176
163
174
172
174
173
174
168
169
171
179
173
172
174
166
166
163
177
158
163
166
178
180
172
162
176
174
165
163
173
174
168
173
165
177
177
177
180
167
177
165
165
161
172
178
175
165
167
171
178
172
175
172
173
163
160
174
174
173
174
161
178
177
172
181
173
179
180
166
175
165
172
177
166
162
173
172
174
177
179
170
172
171
172
173
172
177
173
178
173
170
175
178
169
172
175
172
173
172
167
167
164
173
172
171
172
171
172
170
172
170
166
171
172
174
177
169
170
166
172
164
178
172
170
171
173
172
175
175
168
175
173
178
175
164
164
172
165
178
165
163
165
165
165
164
166
165
166
165
152
151
167
170
161
171
154
154
154
151
168


In [120]:
# my clusters
sns.scatterplot(X, np.linspace(0, 1, len(clusters)), 
                hue=clusters, palette='bright')
plt.title('Plotted from my clusters!')
plt.show();

# sk-learn clusters
sns.scatterplot(X, np.linspace(0, 1, len(clusters)), 
                hue=sk_clusters, palette='bright')
plt.title('Plotted from sk-learn clusters!')
plt.show();


Exception: Data must be 1-dimensional

In [87]:
clusters

923